In [1]:
import logging
import os
import sys

from htools import *

In [41]:
class LoggerMixin:
    """Mixin class that configures and returns a logger.
    
    Examples
    --------
    class Foo(LoggerMixin):
    
        def __init__(self, a, log_file):
            self.a = a
            self.log_file = log_file
            self.logger = self.get_logger(log_file)
        
        def walk(self, location):
            self.logger.info(f'walk received argument {location}')
            return f'walking to {location}'
    """
    
    def get_logger(self, fname=None, fmode='a', level='info', 
                   fmt='%(asctime)s [%(levelname)s]: %(message)s'):
        """
        Parameters
        ----------
        fname: str or None
            If provided, this will be the path the logger writes to. 
            If left as None, logging will only be to stdout.
        fmode: str
            Logging mode when using a log file. Default 'a' for 
            'append'. 'w' will overwrite the previously logged messages.
        level: str
            Minimum level necessary to log messages. 
            One of ('debug', 'info', 'warning', 'error')
        fmt: str
            Format that will be used for logging messages. The logging 
            module has a specific 
            
        Returns
        -------
        logging.logger
        """

        # When working in Jupyter, need to reset handlers.
        # Otherwise every time we run a cell creating an 
        # instance of the logged class, the list of handlers will grow.
        logger = logging.getLogger(type(self).__name__)
        logger.handlers.clear()
        logger.setLevel(getattr(logging, level.upper()))
        
        # handler.basicConfig() doesn't work in Jupyter..
        formatter = logging.Formatter(fmt)
        handlers = [logging.StreamHandler(sys.stdout)]
        if fname: 
            handlers.append(logging.FileHandler(fname, fmode))
        for handler in handlers:
            handler.setFormatter(formatter)
            logger.addHandler(handler)
        
        return logger

In [25]:
@auto_repr
class Foo(LoggerMixin):
    
    def __init__(self, a, log_file=None):
        self.a = a
        self.log_file = log_file
        self.logger = self.get_logger(log_file)
        
    def walk(self, b):
        self.logger.info(f'logger msg {b}')
        return f'WALKING {self.a}, {b}'

In [32]:
f = Foo([3, 4, 5], os.path.join('..', 'data', 'foo.log'))
f

Foo(a=[3, 4, 5], log_file='../data/foo.log')

In [33]:
_ = f.walk(6)

2020-02-01 09:58:50,187 [INFO]: logger msg 6


In [34]:
!cat ../data/foo.log

2020-02-01 09:58:50,187 [INFO]: logger msg 6


In [35]:
f.walk(11)

2020-02-01 09:58:50,941 [INFO]: logger msg 11


'WALKING [3, 4, 5], 11'

In [36]:
!cat ../data/foo.log

2020-02-01 09:58:50,187 [INFO]: logger msg 6
2020-02-01 09:58:50,941 [INFO]: logger msg 11


In [37]:
f = Foo([9])

In [38]:
f.walk(100)

2020-02-01 09:59:12,566 [INFO]: logger msg 100


'WALKING [9], 100'

In [39]:
!cat ../data/foo.log

2020-02-01 09:58:50,187 [INFO]: logger msg 6
2020-02-01 09:58:50,941 [INFO]: logger msg 11
